---
## SQL Agents
Fourth approach, continued from SQLBasedChains.ipynb.

Agent created from *create_sql_agent*, contains *SQLDatabaseToolkit*.
* It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table)
* It can recover from errors by running a generated query, catching the traceback and regenerating it correctly


In [ ]:
from langchain.agents.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.sql_database import SQLDatabase
from langchain.llms import GPT4All


memory = ConversationBufferMemory(memory_key = 'history' , input_key = 'input')
llm = GPT4All(model="../models/ggml-model-gpt4all-falcon-q4_0.bin")
db = SQLDatabase.from_uri("sqlite:///../db/retention-sqlite.db")


suffix = """Begin!

Relevant pieces of previous conversation:
{history}
(You do not need to use these pieces of information if not relevant)

Question: {input}
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
{agent_scratchpad}
"""
executor = create_sql_agent(
    llm = llm,
    toolkit = SQLDatabaseToolkit(db=db, llm=llm),
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    input_variables = ["input", "agent_scratchpad", "history"],
    suffix = suffix, # must have history as variable,
    agent_executor_kwargs = {'memory':memory},
    verbose=True,
)

In [ ]:
# NOTE:
# Local GPT4All generate ill-formatted answers
executor.run("What are the tables in the database?")